# End-to-end `scivision` pipeline for a pretrained/prebuilt model for tree crown detection

This notebook demonstrates `scivision` to load `detectreeRGB`, a pretrained model to predict the location and extent of tree crowns from a top-down RGB image, captured by drone, aircraft or satellite. `detectreeRGB` was implemented in python 3.8 using Mask R-CNN deployed from [detectron2](https://github.com/facebookresearch/detectron2/blob/main/docs/tutorials/install.md) library to delineate tree crowns accurately.
Further details of the `detectreeRGB` can be found in [the original model repository](https://github.com/shmh40/detectreeRGB).

The code of this notebook let `scivision` to fetch input data from:
https://github.com/acocac/scivision-forest-datasets/drone_tropics.yml

and load the pretrained `detectreeRGB` from:
https://github.com/acocac/scivision-treecrown-plugin

## Data catalogue

### Load libraries

In [ ]:
from scivision.io import load_dataset, load_pretrained_model
from intake_zenodo_fetcher import download_zenodo_files_for_entry

### Load images

In [ ]:
cat = load_dataset('https://github.com/acocac/scivision-forest-datasets/drone_tropics.yml')

In [ ]:
download_zenodo_files_for_entry(cat['sepilok_rgb'], force_download=False)

In [ ]:
tc_rgb  = cat["sepilok_rgb"].to_dask()

## Model catalogue

### Install model

In [ ]:
# Load model
scivision_yml = 'https://github.com/acocac/scivision-treecrown-plugin/.scivision-config.yaml'
model = load_pretrained_model(scivision_yml, allow_install=True)

In [ ]:
# let's explore the model object
model

### Load libraries

In [ ]:
!pip install opencv-python-headless

In [ ]:
import numpy as np
import cv2
from detectron2.utils.visualizer import Visualizer, ColorMode
from PIL import Image
from IPython.display import display

## Data preparation

In [ ]:
minx = 602500
miny = 646600

R = tc_rgb[0]
G = tc_rgb[1]
B = tc_rgb[2]

# stack up the bands in an order appropriate for saving with cv2, then rescale to the correct 0-255 range for cv2

# you will have to change the rescaling depending on the values of your tiff!
rgb = np.dstack((R, G, B))  # BGR for cv2
rgb_rescaled = 255 * rgb / 65535  # scale to image

# save this as png, named with the origin of the specific tile - change the filepath!
filepath = 'tile_' + str(minx) + '_' + str(miny) + '.png'
cv2.imwrite(filepath, rgb_rescaled)

X = cv2.imread(filepath)

## Prediction and visualisation

In [ ]:
y = model.predict(X)

v = Visualizer(X[:, :, ::-1], scale=1.5, instance_mode=ColorMode.IMAGE_BW)   # remove the colors of unsegmented pixels
v = v.draw_instance_predictions(y["instances"].to("cpu"))
image = cv2.cvtColor(v.get_image()[:, :, :], cv2.COLOR_BGR2RGB)
display(Image.fromarray(image))